Note that data is already downloaded in separate files to data/irrelevant from Kacper's work.

In [1]:
import pandas as pd
import os, json
from tqdm.notebook import tqdm

tqdm.pandas()

In [2]:
files = os.listdir("../../../data/irrelevant/")

data = {}
errors = []

for i,file in enumerate(tqdm(files)):
    with open('../../../data/irrelevant/' + file,'r') as f:
        try:
            data[i] = json.loads(f.read())
        except:
            errors.append(file)
            continue

data = pd.DataFrame.from_dict(data, orient='index',columns=['URL', 'ExtractedTextUntokenized','RawData'])
data.head()

  0%|          | 0/2132 [00:00<?, ?it/s]

,URL,ExtractedTextUntokenized,RawData
0,https://www.deere.com/assets/pdfs/region-2/our...,"1/2024 Learning, fun and farming h t n o m e h...",NaN
1,https://www.dcmshriram.com/images/H2ECComplian...,th Ref. No. :SFC/EPC/2.4/174A Date : 30 June 2...,NaN
2,https://ncs.go.ug/files/NCS%20ANNUAL%20REPORT%...,ANNUAL REPORT 2022/2023 NCS CORE VALUES Honest...,NaN
3,https://dellrapidschamber.com/wp-content/uploa...,May Newsletter We exist to support local busin...,NaN
4,http://debejyo.com/publications/spie10_chakrab...,Active Learning Data Selection for Adaptive On...,NaN


In [3]:
for file in errors:
    try:
        os.remove('data/irrelevant/' + file)
    except FileNotFoundError:
        continue
print('Removed', len(errors), 'broken files.')

Removed 0 broken files.


In [4]:
# get record where ExtractedTextUntokenized is empty or RawData has a value
pdfs = data[(data['ExtractedTextUntokenized'].isnull())]
pdfs.head()

,URL,ExtractedTextUntokenized,RawData


In [5]:
import pymupdf
import requests

def extract_pdf_text(url):
    r = requests.get(url)
    try:
        pdf = pymupdf.open(stream=r.content, filetype='pdf')
    except Exception as e:
        print('Error opening PDF: ', url)
        print(e)
        return None
    return '\n\n'.join([pdf.get_page_text(i) for i in range(len(pdf))])

In [6]:
pdfs['ExtractedTextUntokenized'] = pdfs['URL'].progress_apply(extract_pdf_text)

0it [00:00, ?it/s]

In [7]:
data.update(pdfs)
data.dropna(subset=['ExtractedTextUntokenized'], inplace=True)
# drop empty strings too
data = data[data['ExtractedTextUntokenized'] != '']

In [8]:

# renaming columns
write_data = data[['URL', 'ExtractedTextUntokenized']]
write_data.columns = ['url', 'text']
write_data['class'] = 'irrelevant'

write_data

,url,text,class
0,https://www.deere.com/assets/pdfs/region-2/our...,"1/2024 Learning, fun and farming h t n o m e h...",irrelevant
1,https://www.dcmshriram.com/images/H2ECComplian...,th Ref. No. :SFC/EPC/2.4/174A Date : 30 June 2...,irrelevant
2,https://ncs.go.ug/files/NCS%20ANNUAL%20REPORT%...,ANNUAL REPORT 2022/2023 NCS CORE VALUES Honest...,irrelevant
3,https://dellrapidschamber.com/wp-content/uploa...,May Newsletter We exist to support local busin...,irrelevant
4,http://debejyo.com/publications/spie10_chakrab...,Active Learning Data Selection for Adaptive On...,irrelevant
...,...,...,...
2127,https://deadliestwebattacks.com/assets/devops-...,"DevOps Is Automation, DevSecOps Is People Mike...",irrelevant
2128,https://www.daytonsuperior.com/docs/default-so...,Yeti-Anchor® Precast & Tilt-Up TECHNICAL DATA ...,irrelevant
2129,https://dce-uae.com/wp-content/uploads/2024/04...,MAŁOPOLSKA 14 UNESCO World Heritage List sites...,irrelevant
2130,https://www.dekosrl.com/wp-content/uploads/202...,"CHAPTER ONE TheCompany 4 Mission: know-how, pe...",irrelevant


In [ ]:
# write to json
write_data.to_json('../../../data/labelled/irrelevant.json', orient='records', indent=4)

